In [178]:
import re
from tika import parser
from os import listdir
from os.path import isfile, join
import os
import pandas as pd
from datetime import datetime, date

In [179]:
source_folder = "../data/pdf/"
output_folder = "../data/"

In [181]:
# 폴더내 pdf파일을 파싱하여 Excel 형태로 저장
def pdf2txt (source_folder, output_folder):
    # 지정 폴더 내 파일 목록 조회 (파일만)
    pdf_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    
    result = []
    for pdf in pdf_files :
        time = pdf.split('(')
        time = time[2][:-5]
        
        pdf_filepath = source_folder + pdf
        # pdf 파일을 text로 변환
        parsedText = parser.from_file(pdf_filepath)["content"]
        new_dict = {
            'time' : time,
            'text' : parsedText
        }
        result.append(new_dict)

    df = pd.DataFrame(result)

    temp_name = f"금통위의사록(텍스트파일).xlsx"
    txt_name = os.path.join(output_folder, temp_name)
    excel_writer = pd.ExcelWriter(txt_name, engine = 'openpyxl')

    df.to_excel(excel_writer)
    excel_writer.save()
    
pdf2txt(source_folder, output_folder)

In [182]:
# 의사록 time 컬럼 datetime으로 변환
minutes = pd.read_excel('../data/금통위의사록(텍스트파일).xlsx', usecols=[1, 2])
minutes['time'] = minutes['time'].astype(str)

date_list = []
for time in minutes['time']:
    date_list.extend(re.findall('[0-9]{4}.[0-9]{1,2}.[0-9]{1,2}', time))
minutes['time'] = date_list
minutes['time'] = pd.to_datetime(minutes['time'], format='%Y.%m.%d')

minutes

,time,text
0,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,2005-06-23,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,2005-07-07,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,2005-08-11,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,2005-09-08,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
...,...,...
365,2023-01-13,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
366,2023-02-23,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
367,2023-03-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
368,2023-03-23,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [183]:
# 연도가 2009~2021년인 의사록 filtering
minutes = minutes[(minutes['time'].dt.year >= 2009) & (minutes['time'].dt.year <= 2021)]
minutes

,time,text
76,2009-04-23,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
77,2009-04-30,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
78,2009-05-12,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
79,2009-05-21,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
80,2009-06-11,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
...,...,...
336,2021-01-28,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
337,2021-02-25,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
338,2021-03-11,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
339,2021-03-25,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [186]:
# 텍스트 전처리 함수
def text_filtering(text):
    text = re.sub('\\n{2,}','', text)
    text = re.sub('- \d+ -', '', text)
    text = re.sub('―|｢|｣|-|[․/→←+]', '', text)
    text = re.sub('^.*hwp*\w*[A-Za-z]', '', text)
    return text

minutes['text'] = minutes['text'].apply(text_filtering)
minutes

C:\Users\leoni\AppData\Local\Temp\ipykernel_17004\1406083749.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minutes['text'] = minutes['text'].apply(text_filtering)


,time,text
76,2009-04-23,금융통화위원회 의사록2009년도 제 차 회의10일 자1. 년 월 일 목2009 4 ...
77,2009-04-30,금융통화위원회 의사록2009년도 제 차 회의11일 자1. 년 월 일 목2009 4 ...
78,2009-05-12,금융통화위원회 의사록2009년도 제 차 회의12일 자1. 년 월 일 목2009 5 ...
79,2009-05-21,금융통화위원회 의사록2009년도 제 차 회의13일 자1. 년 월 일 목2009 5 ...
80,2009-06-11,금융통화위원회 의사록2009년도 제 차 회의14일 자1. 년 월 일 목2009 6 ...
...,...,...
336,2021-01-28,2021년도 제2차금융통화위원회(정기) 의사록한 국 은 행1. 일 자 2021년 1...
337,2021-02-25,2021년도 제4차금융통화위원회(정기) 의사록한 국 은 행1. 일 자 2021년 2...
338,2021-03-11,2021년도 제5차금융통화위원회(정기) 의사록한 국 은 행1. 일 자 2021년 3...
339,2021-03-25,2021년도 제6차금융통화위원회(정기) 의사록한 국 은 행1. 일 자 2021년 3...


In [187]:
minutes.to_excel('의사록_분리.xlsx')